In [2]:
import json
from collections import defaultdict
import operator
from difflib import get_close_matches

In [3]:
data1 = []
for line in open('business.json','r'): 
     data1.append(json.loads(line))

In [4]:
data1[0]

{'address': '4855 E Warner Rd, Ste B9',
 'attributes': {'AcceptsInsurance': True,
  'BusinessAcceptsCreditCards': True,
  'ByAppointmentOnly': True},
 'business_id': 'FYWN1wneV18bWNgQjJ2GNg',
 'categories': ['Dentists',
  'General Dentistry',
  'Health & Medical',
  'Oral Surgeons',
  'Cosmetic Dentists',
  'Orthodontists'],
 'city': 'Ahwatukee',
 'hours': {'Friday': '7:30-17:00',
  'Monday': '7:30-17:00',
  'Thursday': '7:30-17:00',
  'Tuesday': '7:30-17:00',
  'Wednesday': '7:30-17:00'},
 'is_open': 1,
 'latitude': 33.3306902,
 'longitude': -111.9785992,
 'name': 'Dental by Design',
 'neighborhood': '',
 'postal_code': '85044',
 'review_count': 22,
 'stars': 4.0,
 'state': 'AZ'}

### Categories vs Business

In [5]:
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd

In [6]:
CategoryBasedBus = defaultdict(int)
for d in data1:
    for c in d['categories']:
        CategoryBasedBus[c] +=1   

In [7]:
PopularCategory = set()
total_reviews_main = 0
threshold = 50
CategoryBasedBusSorted = {}

CategoryBasedBusSorted = dict(sorted(CategoryBasedBus.items(),key=operator.itemgetter(1),reverse=True))
review_count = 0
for category, review in CategoryBasedBusSorted.items():
    PopularCategory.add(category)
    review_count += review
    if review_count > total_reviews_main*1.0*threshold/(100.0):
        break

In [8]:
top_category_list = []
top_category_reviews = []
count = 0
total_rows_in_plot = 9
for c,r in CategoryBasedBusSorted.items():
    count+=1
    print (c,r)
    top_category_list.append(c)
    top_category_reviews.append(r)
    if count > total_rows_in_plot:
        break

Restaurants 54618
Shopping 27971
Food 24777
Beauty & Spas 17014
Home Services 16205
Health & Medical 14230
Nightlife 12154
Local Services 11232
Automotive 11052
Bars 10563


In [9]:
top_category_list,top_category_reviews, total_reviews_main

(['Restaurants',
  'Shopping',
  'Food',
  'Beauty & Spas',
  'Home Services',
  'Health & Medical',
  'Nightlife',
  'Local Services',
  'Automotive',
  'Bars'],
 [54618, 27971, 24777, 17014, 16205, 14230, 12154, 11232, 11052, 10563],
 0)

In [10]:
plot1 = []
bad_okay_good=[10000, 20000, 30000,50000]
for counter in range(total_rows_in_plot):
    plot1.append({"title":top_category_list[counter],"subtitle":"dummy","ranges":bad_okay_good,"measures":[top_category_reviews[counter],top_category_reviews[counter]+1],"markers":[top_category_reviews[counter]]})
plot1 = json.dumps(plot1)

In [11]:
plot1_data = pd.read_json(plot1)
fig1 = ff.create_bullet(plot1_data, markers="markers", measures='measures',ranges="ranges",subtitles="subtitle",titles="title")
plotly.offline.plot(fig1, filename = "restaurant.html")

'file:///Users/kavyaputluri/Documents/ECE 180/Project/restaurant.html'

### Geographic Distribution

In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [15]:
restaurant_data = []
for d in data1:
    if 'Restaurants' in d['categories']:
        restaurant_data.append(d)

In [16]:
CityBasedBus = defaultdict(int)
PopularCity = set()
total_reviews = 0
threshold = 70
for business in restaurant_data:
    city = business['city']
    review_count = business['review_count']
    CityBasedBus[city] += review_count
    total_reviews += review_count
    
CityBasedBusSorted = dict(sorted(CityBasedBus.items(),key=operator.itemgetter(1), reverse=True))
review_count = 0
for city, review in CityBasedBusSorted.items():
    PopularCity.add(city)
    review_count += review
    if review_count > total_reviews*1.0*threshold/(100.0):
        break

In [104]:
PopularCityListExpanded = []
cityList = list(CityBasedBusSorted.keys())
for city in PopularCity:
    PopularCityListExpanded += get_close_matches(city, cityList,20)
    

In [102]:
popular_city_restaurant_data = [] #Will act as business.json
business_id_to_use = [] #Will be used to generate review.json
for business in restaurant_data:
    city = business['city']
    if city in PopularCityListExpanded:
        popular_city_restaurant_data.append(business)
        business_id_to_use.append(business['business_id'])

In [103]:
total_cities_plot = 200

In [93]:
count = 0
geo_fd = open("geogrpahy.csv","w")
geo_fd.write("name,review,lat,lon\n")
for city, review in CityBasedBusSorted.items():
    try:
        location = geolocator.geocode(city)
        geo_fd.write(city+","+str(review)+","+str(location.latitude)+","+str(location.longitude)+"\n")
    except:
        break
    count += 1
    if count == total_cities_plot:
        break
geo_fd.close()
total_cities_plot = count

In [94]:
count = 0
cities = []
scale = [400,300,200,100,50]
df = pd.read_csv('geogrpahy.csv',encoding = "ISO-8859-1")
df.head()
limits = [(0,4),(4,12),(12,24),(24,40),(40,total_cities_plot)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        marker = dict(
            size = df_sub['review']/scale[i],
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
    )
    cities.append(city)
    

layout = dict(
        title = 'City Based Review Popularity of Restaurants',
        showlegend = True,
        geo = dict(
            scope='usa',
            #projection=dict( type='natural earth' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig2 = dict(data=cities, layout = layout)
plotly.offline.plot(fig2, filename="geography.html")
    

'file:///Users/kavyaputluri/Documents/ECE 180/Project/geography.html'

### Others

In [22]:
restaurant_data = []
for d in data1:
    if 'Restaurants' in d['categories']:
        restaurant_data.append(d)

In [23]:
restaurant_data[234]

{'address': '22578 Lake Shore Blvd',
 'attributes': {'Ambience': {'casual': False,
   'classy': False,
   'divey': False,
   'hipster': False,
   'intimate': False,
   'romantic': False,
   'touristy': False,
   'trendy': False,
   'upscale': False},
  'BikeParking': True,
  'BusinessAcceptsCreditCards': True,
  'BusinessParking': {'garage': False,
   'lot': False,
   'street': False,
   'valet': False,
   'validated': False},
  'GoodForKids': True,
  'GoodForMeal': {'breakfast': True,
   'brunch': True,
   'dessert': False,
   'dinner': False,
   'latenight': False,
   'lunch': True},
  'HasTV': True,
  'NoiseLevel': 'average',
  'OutdoorSeating': False,
  'RestaurantsAttire': 'casual',
  'RestaurantsDelivery': False,
  'RestaurantsGoodForGroups': True,
  'RestaurantsPriceRange2': 1,
  'RestaurantsReservations': False,
  'RestaurantsTableService': True,
  'RestaurantsTakeOut': True,
  'WheelchairAccessible': True,
  'WiFi': 'no'},
 'business_id': 'gchI4BdZ76teMX9BsI2H-w',
 'categories